In [ ]:
class ShapeDetector:
	def __init__(self):
		pass
	def detect(self, c):
		# initialize the shape name and approximate the contour
		shape = "unidentified"
		peri = cv2.arcLength(c, True)
		approx = cv2.approxPolyDP(c, 0.04 * peri, True)
		# if the shape is a triangle, it will have 3 vertices
		if len(approx) == 3:
			shape = "triangle"
		# if the shape has 4 vertices, it is either a square or
		# a rectangle
		elif len(approx) == 4:
			# compute the bounding box of the contour and use the
			# bounding box to compute the aspect ratio
			(x, y, w, h) = cv2.boundingRect(approx)
			ar = w / float(h)
			# a square will have an aspect ratio that is approximately
			# equal to one, otherwise, the shape is a rectangle
			shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"
		# if the shape is a pentagon, it will have 5 vertices
		elif len(approx) == 5:
			shape = "pentagon"
		# otherwise, we assume the shape is a circle
		else:
			shape = "circle"
		# return the name of the shape
		return shape

In [ ]:
import numpy as np

def read_csv(csv_path):
    # Read the CSV file into a NumPy array
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',')
    path_XYs = []
    coordinates = []  # List to store all x and y coordinates

    # Iterate over unique paths identified by the first column
    for i in np.unique(np_path_XYs[:, 0]):
        # Extract points belonging to the current path
        npXYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []

        # Iterate over unique segments identified by the first column of npXYs
        for j in np.unique(npXYs[:, 0]):
            # Extract points belonging to the current segment
            XY = npXYs[npXYs[:, 0] == j][:, 1:]
            XYs.append(XY)
            # Append x and y coordinates to the coordinates list
            for point in XY:
                coordinates.append((int(point[0]), int(point[1])))  

        path_XYs.append(XYs)

    return path_XYs, coordinates

# Example usage
csv_path = r'D:\Desktop\adobe\problems\problems\isolated.csv'
result, coordinates = read_csv(csv_path)
# print("Path XYs:", result)
# print("Coordinates:", coordinates)

# Example usage with OpenCV to create and display the image
import cv2

# Determine the dimensions of the image
max_x = max([x for x, y in coordinates])
max_y = max([y for x, y in coordinates])

# Create an empty (white) image with dimensions based on the max coordinates
height = max_y + 1
width = max_x + 1

# Create an empty (white) image with all pixels set to 255 (white)
image = np.ones((height, width), dtype=np.uint8) * 255

# Set the pixels at the specified coordinates to 0 (black)
for (x, y) in coordinates:
    image[y, x] = 0  # Note that the array indexing is image[row, col] which is (y, x)

# Display the image using OpenCV
cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import imutils
resized = imutils.resize(image, width=300)
ratio = image.shape[0] / float(resized.shape[0])
# convert the resized image to grayscale, blur it slightly,
# and threshold it
gray = image.copy()
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY)[1]
# find contours in the thresholded image and initialize the
# shape detector
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
sd = ShapeDetector()
# loop over the contours
for c in cnts:
	# compute the center of the contour, then detect the name of the
	# shape using only the contour
	M = cv2.moments(c)
	cX = int((M["m10"] / M["m00"]) * ratio)
	cY = int((M["m01"] / M["m00"]) * ratio)
	shape = sd.detect(c)
	# multiply the contour (x, y)-coordinates by the resize ratio,
	# then draw the contours and the name of the shape on the image
	c = c.astype("float")
	c *= ratio
	c = c.astype("int")
	cv2.drawContours(image, [c], -1, (0, 255, 0), 2)
	cv2.putText(image, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX,
		0.5, (255, 255, 255), 2)
	# show the output image
	cv2.imshow("Image", image)
	cv2.waitKey(0)

In [ ]:
import cv2
import numpy as np
import mahotas
def detect_shapes(image):
    # Load and preprocess the image
    gray = image
     # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    cv2.imshow("blurred", blurred)
    cv2.waitKey(0)
     # Apply binary thresholding
    T=mahotas.thresholding.rc(blurred)
    _, thresh = cv2.threshold(blurred, T, 255, cv2.THRESH_BINARY_INV)
    cv2.imshow("thresh",thresh)
    cv2.waitKey(0)
    # Detect edges
    edged = cv2.Canny(thresh, 50, 150)
    cv2.imshow("edged",edged)
    cv2.waitKey(0)
    # Detect contours
    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    print("No. of contours: ", len(contours))
        
    i=0
    for contour in contours:
        if i==0:
            i=1
            continue
        # Approximate the contour
        epsilon = 0.08 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        shape = ""
        if len(approx) == 3:
            shape = "Triangle"
        elif len(approx) == 4:
            (x, y, w, h) = cv2.boundingRect(approx)
            aspectRatio = w / float(h)
            if 0.95 <= aspectRatio <= 1.05:
                shape = "Square"
            else:
                shape = "Rectangle"
        elif len(approx) == 5:
            shape = "Pentagon"
        elif len(approx) > 5:
            (x, y), radius = cv2.minEnclosingCircle(contour)
            area = cv2.contourArea(contour)
            circle_area = np.pi * (radius ** 2)
            if abs(circle_area - area) < 0.1 * area:
                shape = "Circle"
            else:
                ellipse = cv2.fitEllipse(contour)
                shape = "Ellipse"
        print("Shape: ", shape)

        # Draw the detected shape
        if shape == "Triangle":
            cv2.drawContours(image, [approx], -1, (0, 255, 0), 2)
        elif shape == "Square" or shape == "Rectangle":
            (x, y, w, h) = cv2.boundingRect(approx)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        elif shape == "Pentagon":
            cv2.drawContours(image, [approx], -1, (0, 255, 0), 2)
        elif shape == "Circle":
            center = (int(x), int(y))
            cv2.circle(image, center, int(radius), (0, 255, 0), 2)
        elif shape == "Ellipse":
            ellipse = cv2.fitEllipse(contour)
            cv2.ellipse(image, ellipse, (0, 255, 0), 2)
        
        # Optional: label the shape
        cv2.putText(image, shape, (approx[0][0][0], approx[0][0][1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    # Display the result
    cv2.imshow("Detected Shapes", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

detect_shapes(image)
